In [182]:
import json

import pandas as pd
import numpy as np


In [183]:
df = pd.read_csv("generations_example.csv")
df.head()

,prompt,completion,reward,completion_ids,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_915731_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272
0,<|im_start|>user\nExplain following rating of ...,Here is a detailed explanation of each rating ...,-26.562500,"[8420, 374, 264, 11682, 16148, 315, 1817, 1072...",Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-1.562500,0.460938
1,<|im_start|>user\nExplain following rating of ...,Here's a detailed explanation of each rating (...,-23.914062,"[8420, 594, 264, 11682, 16148, 315, 1817, 1072...",Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,1.085938,0.304688
2,<|im_start|>user\nExplain following rating of ...,Here's a summary of the four main parameters i...,-27.203125,"[8420, 594, 264, 12126, 315, 279, 3040, 1887, ...",Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-2.203125,-2.281250
3,<|im_start|>user\nExplain following rating of ...,Here are explanations for each of the 10 stand...,-25.593750,"[8420, 525, 40841, 369, 1817, 315, 279, 220, 1...",Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-0.593750,-0.562500
4,<|im_start|>user\nExplain following rating of ...,Here is a detailed explanation of the rating p...,-26.125000,"[8420, 374, 264, 11682, 16148, 315, 279, 10728...",Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-1.125000,-0.703125


In [184]:
df.__len__()

61168

In [185]:
df_unique = df #df.drop_duplicates(subset=['completion'], keep='first')
df_unique.__len__()

61168

In [186]:
columns = df_unique.columns.tolist()
columns_reward = [col for col in columns if col.startswith('reward_')]
column_max = columns_reward[0]
column_min = columns_reward[1]
column_max, column_min

('reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272',
 'reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_915731_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272')

In [187]:
df_filtered = df_unique[df_unique[column_max] > df_unique['reference_reward']]
len(df_filtered)

28119

In [188]:
df_filtered[column_max].mean(), df_filtered[column_min].mean()

(np.float64(1.8720541704116063), np.float64(0.677565933575378))

In [189]:
uncertainty_reference = np.abs(df_filtered['reference_reward_1'] - df_filtered['reference_reward_2'])
mu_U = uncertainty_reference.mean()
sigma_U = uncertainty_reference.std(ddof=1)

mu_U, sigma_U

(np.float64(1.0674402904033322), np.float64(1.1160521421921694))

In [190]:
uncertainty_adv = np.abs(df_filtered[column_max] - df_filtered[column_min])
z_score = (uncertainty_adv - mu_U) / sigma_U

In [191]:
df_tested = df_filtered[z_score > 1.96]
df_tested.__len__()

1109

In [192]:
df_tested['completion'].str.contains('⚙').sum()

np.int64(17)

In [193]:
df_unique_prompt = df_tested.drop_duplicates(subset=['prompt'], keep='first')
df_unique_prompt.__len__()

561

In [194]:
df_unique_prompt[-10:]

,prompt,completion,reward,completion_ids,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_915731_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272
60321,<|im_start|>user\nWhat is used behind the scen...,"Behind the scene in **Power BI**, the **key in...",3.292969,"[73642, 279, 6109, 304, 3070, 14986, 47053, 97...",Behind the scene for key influencer visual in ...,Behind the scene for key influencers in Power ...,3.523438,3.847656,3.201172,3.953125,0.660156
60353,<|im_start|>user\nwrite a quick review on The ...,Here's a **quick and professional review** of ...,3.612305,"[8420, 594, 264, 3070, 27763, 323, 6584, 3395,...",**Review of *The Haxan Cloak - Excavation*** ...,**Review: *The Haxan Cloak – Excavation*** \n...,1.179688,0.505859,1.852539,3.468750,-0.143555
60416,<|im_start|>user\nwho said the principle (dire...,The **Principle (Directive Principles) of the ...,4.000000,"[785, 3070, 3533, 15996, 320, 62076, 56914, 8,...",The principle (Directive Principles) of State ...,The principle (directive principles) is a set ...,-1.908203,-2.447266,-1.368164,3.218750,-0.781250
60536,<|im_start|>user\ncan u explain IT Revolution ...,Certainly! The **** **** **** ** ** ** ** ** \...,3.314453,"[95456, 0, 576, 3070, 334, 3070, 334, 3070, 33...",Certainly! Here's an explanation of **IT Revol...,Certainly! Here's a structured explanation of ...,2.675781,2.292969,3.056641,3.234375,-0.080078
60631,<|im_start|>user\nWhen did Russia and Japan ma...,Russia and Japan did **not reach peace** after...,4.335938,"[44506, 323, 6323, 1521, 3070, 1921, 5545, 891...","After World War II, Russia and Japan did not o...",Russia and Japan did not make peace after Worl...,-1.637695,-3.685547,0.410889,-1.007812,-5.343750
60721,<|im_start|>user\nCan you come up with an orig...,Absolutely! Here's an **original Magic: The Ga...,3.277344,"[77045, 0, 5692, 594, 458, 3070, 9889, 15485, ...",Absolutely! Here's an original Magic: The Gath...,Sure! Here's an original Magic: The Gathering ...,2.091797,1.000977,3.183594,3.671875,0.394531
60792,<|im_start|>user\nwrite a blog on public healt...,Certainly! Here’s a **blog post** titled: \n-...,3.460938,"[95456, 0, 5692, 748, 264, 3070, 11659, 1736, ...",**Title: Mastering Public Health: Building a S...,**Title: Building a Health Future: Exploring P...,2.960938,2.906250,3.015625,4.562500,1.101562
60848,<|im_start|>user\nwrite letter to the editor t...,Certainly! Below is a **summary letter** writt...,3.283691,"[95456, 0, 21193, 374, 264, 3070, 1708, 6524, ...",**Letter to the Editor** \n**Subject:** Progr...,**Letter to the Editor**\n\n**Subject:** Progr...,2.650391,3.001953,2.298828,3.203125,-0.080566
60881,<|im_start|>user\npx4 Flight mode<|im_end|>\n<...,"**""PX4 Flight Mode"" 用于 **[** Cruise (Cruise Mo...",3.578125,"[334, 1, 73812, 19, 26577, 14562, 1, 220, 1007...",Hello! I'm here to help with your PX4 flight m...,The `px4` (Pilot's Flight Mode) is a specific ...,2.832031,3.554688,2.107422,3.859375,0.281250
61086,<|im_start|>user\nI want you to pretend to be ...,Sure! Here's your **revised article** in the s...,3.531250,"[39814, 0, 5692, 594, 697, 3070, 265, 25379, 4...",Here is your updated opinion article with the ...,**ChatLinker** \nWe Liberals Have Overreacted...,0.015625,-0.249512,0.280762,2.515625,-1.015625


In [195]:
with open('data/train.json', 'r') as f:
    train = json.load(f)

In [196]:
list(train[1000].keys())

['chosen',
 'rejected',
 'preference_strength',
 'does_gold_agree_with_original',
 'chosen_reward',
 'rejected_reward',
 'reference_response_1',
 'reference_response_2',
 'reference_reward',
 'reference_reward_1',
 'reference_reward_2']

In [208]:
df_unique_prompt.loc[:, 'best_reference'] = np.where(
    df_unique_prompt['reference_reward_1'] > df_unique_prompt['reference_reward_2'],
    df_unique_prompt['reference_response_1'],
    df_unique_prompt['reference_response_2']
)

df_unique_prompt.loc[:, 'chosen_reward'] = np.where(
    df_unique_prompt['reference_reward_1'] > df_unique_prompt['reference_reward_2'],
    df_unique_prompt['reference_reward_1'],
    df_unique_prompt['reference_reward_2']
)

df_unique_prompt.loc[:, 'rejected_reward'] = df_unique_prompt[column_max]

/tmp/ipykernel_57100/4143826914.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'rejected_reward'] = df_unique_prompt[column_max]


In [198]:
train[1000]

{'chosen': [{'content': 'why protein structure predicts its function',
   'role': 'user'},
  {'content': "The structure of a protein determines its function because the shape of the protein dictates its interactions with other molecules. The three-dimensional structure of a protein is determined by the sequence of amino acids in its chain. Each amino acid has a unique side chain that protrudes from the main chain of the protein, and these side chains can interact with other molecules to form bonds, recognize specific targets, or catalyze chemical reactions.\n\nThe shape of a protein is determined by the interactions between the side chains of the amino acids in its chain. These interactions can be hydrogen bonds, salt bridges, or van der Waals forces, among others. The specific arrangement of these interactions determines the overall shape of the protein and its ability to perform its function.\n\nFor example, enzymes are proteins that catalyze chemical reactions in the body. The three

In [210]:
df_unique_prompt.loc[:, 'chosen'] = df_unique_prompt['prompt'] + df_unique_prompt['best_reference']

df_unique_prompt.loc[:, 'rejected'] = df_unique_prompt['prompt'] + df_unique_prompt['completion']

df_unique_prompt.loc[:, 'does_gold_agree_with_original'] = True
df_unique_prompt.loc[:, 'preference_strength'] = df_unique_prompt[column_max] - df_unique_prompt[column_min]
df_cleaned = df_unique_prompt.drop([
    'prompt',
    'completion',
    'best_reference',
    column_max,
    column_min,
    'completion_ids',
], axis=1)

df_cleaned

/tmp/ipykernel_57100/4181023931.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'preference_strength'] = df_unique_prompt[column_max] - df_unique_prompt[column_min]


,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,chosen,rejected,does_gold_agree_with_original,best_reference_reward,chosen_reward,rejected_reward,preference_strength
529,3.355469,A **Cooperative Process** is a type of process...,A **Cooperative Process** in the context of **...,0.053711,-2.083984,2.191406,<|im_start|>user\nDefine a Cooperative Process...,<|im_start|>user\nDefine a Cooperative Process...,True,2.191406,2.191406,0.183594,3.355469
538,4.429688,**Deck List: Eldrin's Fury** \n*Strategy: A p...,**Deck List: Golgari Eldoria — A Tribal Unity ...,-1.621094,-0.323975,-2.917969,<|im_start|>user\nYou are a writer for a blog ...,<|im_start|>user\nYou are a writer for a blog ...,True,-0.323975,-0.323975,-1.226562,4.429688
792,4.048828,**Title: Master Chief: The IKEA Building Works...,"**Title: ""Building the IKEA Furniture: A Maste...",-0.168213,-1.173828,0.837402,<|im_start|>user\nwrite a script of Master Chi...,<|im_start|>user\nwrite a script of Master Chi...,True,0.837402,0.837402,-0.107422,4.048828
993,3.343750,Here is a comprehensive 9-month Python program...,Here is a **comprehensive step-by-step guide**...,0.799316,1.075195,0.523438,<|im_start|>user\nAct as a personalized learni...,<|im_start|>user\nAct as a personalized learni...,True,1.075195,1.075195,2.187500,3.343750
1196,-3.945312,"THE SOLDIERS CREED IS: **""A TALE OF TWO WIVES ...","THE SOLDIERS CREED: ""I AM THE ONE.""",-3.392578,-4.886719,-1.898438,<|im_start|>user\nIs classical conditioning th...,<|im_start|>user\nIs classical conditioning th...,True,-1.898438,-1.898438,-1.820312,-3.945312
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60721,3.277344,Absolutely! Here's an original Magic: The Gath...,Sure! Here's an original Magic: The Gathering ...,2.091797,1.000977,3.183594,<|im_start|>user\nCan you come up with an orig...,<|im_start|>user\nCan you come up with an orig...,True,3.183594,3.183594,3.671875,3.277344
60792,3.460938,**Title: Mastering Public Health: Building a S...,**Title: Building a Health Future: Exploring P...,2.960938,2.906250,3.015625,<|im_start|>user\nwrite a blog on public healt...,<|im_start|>user\nwrite a blog on public healt...,True,3.015625,3.015625,4.562500,3.460938
60848,3.283691,**Letter to the Editor** \n**Subject:** Progr...,**Letter to the Editor**\n\n**Subject:** Progr...,2.650391,3.001953,2.298828,<|im_start|>user\nwrite letter to the editor t...,<|im_start|>user\nwrite letter to the editor t...,True,3.001953,3.001953,3.203125,3.283691
60881,3.578125,Hello! I'm here to help with your PX4 flight m...,The `px4` (Pilot's Flight Mode) is a specific ...,2.832031,3.554688,2.107422,<|im_start|>user\npx4 Flight mode<|im_end|>\n<...,<|im_start|>user\npx4 Flight mode<|im_end|>\n<...,True,3.554688,3.554688,3.859375,3.578125


In [211]:
df_cleaned.columns.tolist()

['reward',
 'reference_response_1',
 'reference_response_2',
 'reference_reward',
 'reference_reward_1',
 'reference_reward_2',
 'chosen',
 'rejected',
 'does_gold_agree_with_original',
 'best_reference_reward',
 'chosen_reward',
 'rejected_reward',
 'preference_strength']

In [215]:
# make dir
import os
if not os.path.exists('out'):
    os.makedirs('out')
# save the cleaned DataFrame to a JSON file
df_cleaned.to_json('out/train.json', orient='records', lines=False, force_ascii=False)

In [216]:
# read the saved JSON file to verify
with open('out/train.json', 'r') as f:
    train_data = json.load(f)

train_data[100]

{'reward': -4.009765625,
 'reference_response_1': "CUDA (Compute Unified Device Architecture) has several memory types, which are used for different purposes and performance characteristics. Here's a summary of the memory types:\n\n### 1. **Host Memory**\n   - **Purpose**: Communicates directly with the GPU.\n   - **Access**: Host can read from and write to host memory.\n   - **Limitations**: Not part of the GPU memory, so it's not accessible directly from the GPU.\n\n### 2. **Device Memory**\n   - **Purpose**: Used by the GPU to store data.\n   - **Access**: GPU can read from and write to device memory.\n   - **Types**: \n     - **Global Memory**: Shared among all GPUs.\n     - **Local Memory**: Used for small, specific tasks (e.g., in CUDA kernels).\n     - **Global Memory** (used by all GPUs in a multi-GPU setup).\n     - **Local Memory** (used for small tasks).\n     - **Device Memory** (used for specific GPU tasks).\n\n### 3. **Global Memory**\n   - **Purpose**: Shared among all G